In [2]:
import videoio
from PIL import Image
import numpy as np
import tensorflow as tf

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [4]:
def get_video_path(dataset, clazz, video_id):
    return f'/home/dantas/kinetics/{dataset}/{clazz}/{video_id}.mp4'

In [5]:
DATASET = 'train'
CLAZZ = 'ice climbing'
VIDEO_ID = '-2jPR5x7kLw'

In [6]:
video_path = get_video_path(DATASET, CLAZZ, VIDEO_ID)

In [7]:
frames = videoio.videoread(video_path, output_resolution=(299, 299))

In [8]:
frames = np.array(frames)

In [9]:
frames.shape

(300, 299, 299, 3)

In [10]:
img = Image.fromarray(frames[1], 'RGB')
img.show()

In [11]:
model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

In [21]:
result = model(frames[:128])

In [22]:
result.shape

TensorShape([128, 8, 8, 2048])